In [ ]:
#!pip install spacy pyLDAvis

In [ ]:
#!python -m spacy download pt_core_news_sm
#!pip install matplotlib
#!pip install nltk
#!pip install sklearn

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import spacy
import nltk
import re
from nltk import word_tokenize
from nltk.util import ngrams
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

In [ ]:
nltk.download('punkt')

# CONJUNTO DE DADOS

In [ ]:
news = pd.read_csv('articles.csv')
news.head()

In [ ]:
turismo = news[news['category'] == 'turismo']
turismo.info()

In [ ]:
def limpa_texto(texto):
    c = re.sub(r'\n', ' ', texto)
    c = re.sub(r'\r', ' ', c)

    c = c.replace(':', '')
    c = c.replace('/', '')
    c = re.sub(r'R\$', '', c) # Removendo moeda
    c = re.sub(r'\W', '', c) # Removendo alfanúmerico

    c = re.sub(r'\s+', ' ', c) # Removendo multiplo espaço
    c = re.sub(r'^\s+', '', c) # Limpando espaço no início
    c = re.sub(r'\s+$', '', c) # Limpando espaço no final

    return c.lower()



In [ ]:
from spacy.lang.pt import Portuguese

In [ ]:
STOP_WORDS = spacy.lang.pt.stop_words.STOP_WORDS

In [ ]:
def remove_stop(texto):
    doc = nlp_pt(texto)
    return " ".join(x.text for x in doc if x.text not in STOP_WORDS)

In [ ]:
def limpa_texto2(texto):
    doc = nlp_pt(texto)
    return " ".join(x.lemma_ for x in doc if x.text not in STOP_WORDS)

# Análise Exploratória

1. número de palavras no título das notícias

In [ ]:
turismo['title_n_words'] = turismo['title'].apply(lambda x : len(x.split()))

In [ ]:
turismo.head()

In [14]:
turismo.describe()

,title_n_words
count,1903.000000
mean,10.923805
std,1.812440
min,4.000000
25%,10.000000
50%,11.000000
75%,12.000000
max,20.000000


2. Número de caracteres no título das notícias

In [15]:
turismo['title_n_chars'] = turismo['title'].apply(lambda x : len(x.replace(" ", "")))

C:\Users\filip\.pyenv\pyenv-win\versions\3.7.2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
turismo.head()

,title,text,date,category,subcategory,link,title_n_words,title_n_chars
381,Latam inaugura voo direto de 15 horas entre Ch...,Animem-se os que pretendem cruzar o Pacífico e...,2017-06-10,turismo,NaN,http://www1.folha.uol.com.br/turismo/2017/10/1...,11,51
414,Turismo de Las Vegas deve se recuperar em mese...,O massacre em Las Vegas afetará o turismo da c...,2017-06-10,turismo,NaN,http://www1.folha.uol.com.br/turismo/2017/10/1...,11,54
522,Brasileiro cria app para economizar no passeio...,"Com cerca de 50 mil visitantes por dia, Veneza...",2017-05-10,turismo,NaN,http://www1.folha.uol.com.br/turismo/2017/10/1...,11,58
602,AirBnB passa a fazer reserva de restaurantes,A plataforma de hospedagem Airbnb anunciou que...,2017-05-10,turismo,NaN,http://www1.folha.uol.com.br/turismo/2017/10/1...,7,38
609,Museus lembram quatro décadas de opressão sovi...,Nos países que viveram sob a órbita da União S...,2017-05-10,turismo,NaN,http://www1.folha.uol.com.br/turismo/2017/10/1...,10,62


In [17]:
turismo.describe()

,title_n_words,title_n_chars
count,1903.000000,1903.000000
mean,10.923805,55.666842
std,1.812440,6.309523
min,4.000000,25.000000
25%,10.000000,52.000000
50%,11.000000,57.000000
75%,12.000000,60.000000
max,20.000000,87.000000
